# Simplex Method in tableau format

Below is defined a function which comprises the Simplex Method under the table solution approach to solve Linear Programming Problems where we have to minimize or maximize an objective function under some restrictions.

In [1]:
def simplex_tableau(tableau, basic_variables, non_basic_variables, itera = 1):
    
    # Printing our initial parameters for this iteration
    print("\n\n\033[1mIteration ", itera, "\033[0m")
    print("[\033[94m x_B \033[0m, \033[91m x_N \033[0m] = [ \033[94m"
          + ', '.join(basic_variables) + "\033[0m, \033[91m" + ', '.join(non_basic_variables) + "\033[0m ]\n")
    
    # Extracting elements from the tableau of the current iteration
    m = np.shape(tableau)[0] - 1
    n = np.shape(tableau)[1] - 2
    one_array = np.array([tableau[0, 0]])
    zero_array_h = np.ravel(tableau[0, 1:m + 1])
    zero_array_v = tableau[1:m + 1, 0].reshape(1, m)
    identity_matrix = tableau[1:, 1:m + 1]
    ZXN = np.ravel(tableau[0, m + 1:n + 1])
    XBXN = tableau[1:, m + 1:n + 1]
    ZRHS = np.array([tableau[0, n + 1]])
    XBRHS = (tableau[1:, n + 1]).reshape(1, m)
    b_ = np.concatenate((np.ravel(XBRHS), np.zeros(n - m)))
    
    print("\nEntry tableau")
    print(tableau)
    
    z_minus_c_max = max(ZXN)
    k = np.argmax(ZXN)
    
    # Validating if we stop or not the optimization
    if z_minus_c_max <= 0:   # We stop
        
        # Printing the final results
        print("\n\n\033[1mOptimality reached\033[0m")
        print("\nThe optimal BFS is")
        results_ls = [str(round(i, 4)) for i in b_]
        print("\033[92m[ " + ', '.join(basic_variables + non_basic_variables) + " ] = [ " + ', '.join(results_ls) + " ]\033[0m")
        perf_z = ZRHS[0]
        print("\nWith performance z =", perf_z)
        
    else:   # We continue
        
        # Calculating and printing the y_ki's
        y_k = np.ravel(XBXN[:, k])
        
        print("\nk =", k + 1, "-> column no.", k + 1, "of x_N part (", non_basic_variables[k], ")")
        print("y_k")
        print(y_k)
        
        # Analyzing if the optimal BFS is or not boundable by the condition y_k > 0
        flag = True
        aux_counter = 0
        for y_i in y_k:
            if y_i > 0:
                aux_counter = aux_counter + 1      
        if aux_counter == 0:
            flag = False

        if flag:   # Boundable
            
            # Calculating and printing x_k by the minimum quotient (current BFS divided by the y_k > 0) with its index r
            quot_ls = list()
            index_r_ls = list()
            for i in range(m):
                if y_k[i] > 0:
                    quot_ls.append(b_[i] / y_k[i])
                    index_r_ls.append(i)
            quot_arr = np.array(quot_ls)
            index_r_arr = np.array(index_r_ls)
            r = index_r_arr[np.argmin(quot_arr)]
            x_Br = min(quot_arr)
            print("\nr =", r + 1, "-> column no.", r + 1, "of x_B part (", basic_variables[r], ")")
            print("x_Br")
            print(x_Br)
            
            # Pivoting
            pivot = y_k[r]
            print("\npivot =", pivot)
            
            tableau[r + 1, :] = tableau[r + 1, :] / pivot
            for i in range(m + 1):
                if i != r + 1:
                    tableau[i, :] = tableau[i, :] - tableau[i, m + k + 1] * tableau[r + 1, :]
            
            print("\nPivoted tableau")
            print(tableau)
            # Exchanging the indexes
            print("\n\033[94m", non_basic_variables[k], "enters\033[0m and \033[91m", basic_variables[r], "leaves\033[0m the basis")
            aux_vars = basic_variables[r]
            basic_variables[r] = non_basic_variables[k]
            non_basic_variables[k] = aux_vars
            
            # Recursive call
            simplex_tableau(tableau, basic_variables, non_basic_variables, itera + 1)

        else:   # Not boundable
            
            print("\n\n\033[1mOptimization process stopped :(\033[0m")
            print("\nThe optimal BFS is not boundable")
            

This approach of the Simplex Method needs to preprocess our initial data in order to adjust it into a a particular tableau with the below entries:

|         | $z$ | $x_{B}$ | $x_{N}$                | $RHS$         |
|:---:|:---:|:---:|:---:|:---:|
| $z$     | $1$ | $0$     | $c_{b}B^{-1}N - c_{N}$ | $c_b \bar{b}$ |
| $X_{B}$ | $0$ | $I$     | $B^{-1}N$              | $\bar{b}$     |

That if we break it down we get the following equivalent tableau

|          | $z$      | $x_1$      | $x_2$      | $...$ | $x_n$      | $x_{n+1}$         | $...$ | $x_{n+m}$         | $RHS$         |
|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|
| $z$      | $1$      | $z_1-c_1$  | $z_2-c_2$  | $...$ | $z_n-c_n$  | $z_{n+1}-c_{n+1}$ | $...$ | $z_{n+m}-c_{n+m}$ | $c_b \bar{b}$ |
| $X_{B1}$ | $0$      | $Y_{1, 1}$ | $Y_{1, 2}$ | $...$ | $Y_{1, n}$ | $Y_{1, n+1}$      | $...$ | $Y_{1, n+m}$      | $\bar{b_1}$        |
| $X_{B2}$ | $0$      | $Y_{2, 1}$ | $Y_{2, 2}$ | $...$ | $Y_{2, n}$ | $Y_{2, n+1}$      | $...$ | $Y_{2, n+m}$      | $\bar{b_2}$          |
| $\vdots$ | $\vdots$ | $\vdots$   | $\vdots$   |       | $\vdots$   | $\vdots$          |       | $\vdots$          | $\vdots$      |
| $X_{Bm}$ | $0$      | $Y_{m, 1}$ | $Y_{m, 2}$ | $...$ | $Y_{m, n}$ | $Y_{m, n+1}$      | $...$ | $Y_{m, n+m}$      | $\bar{b_m}$          |

In [2]:
import numpy as np
import copy

# Settings for printing numpy arrays
np.set_printoptions(precision=4, edgeitems=10)
np.core.arrayprint._line_width = 180

In [3]:
A = np.matrix([[1, 1, 1, 1, 0, 0], [2, 1, -1, 0, 1, 0], [-1, 3, 0, 0, 0, 1]])
b = np.array([12, 6, 9])
c = np.array([-1, 2, -1, 0, 0, 0])
variables = ["x1", "x2", "x3", "x4", "x5", "x6"]

In [4]:
m, n = np.shape(A)
basic_indexes = range(m, n)
non_basic_indexes = range(m)
basic_variables = variables[m:n]
non_basic_variables = variables[:m]

B = A[:m, basic_indexes]
N = A[:m, non_basic_indexes]
c_b = c[basic_indexes]
c_n = c[non_basic_indexes]
b_ = np.linalg.inv(B) @ b

one_array = np.ones(1, dtype=int)
zero_array_h = np.ravel(np.zeros(m, dtype=int))
zero_array_v = zero_array_h.reshape(1, m)
identity_matrix = np.eye(m)
ZXN = np.ravel((c_b @ np.linalg.inv(B) @ N) - c_n)
XBXN = np.linalg.inv(B) @ N
ZRHS = np.full((1), c_b @ np.ravel(b_))
XBRHS = b_.reshape(1, m)

tableau = np.vstack((np.concatenate((one_array, zero_array_h, ZXN, ZRHS), axis = 0).reshape(1, n + 2),
              np.concatenate((zero_array_v.T, identity_matrix, XBXN, XBRHS.T), axis = 1)))

In [5]:
tableau

matrix([[ 1.,  0.,  0.,  0.,  1., -2.,  1.,  0.],
        [ 0.,  1.,  0.,  0.,  1.,  1.,  1., 12.],
        [ 0.,  0.,  1.,  0.,  2.,  1., -1.,  6.],
        [ 0.,  0.,  0.,  1., -1.,  3.,  0.,  9.]])

In [6]:
simplex_tableau(copy.deepcopy(tableau), copy.deepcopy(basic_variables), copy.deepcopy(non_basic_variables))



Iteration  1 
[ x_B ,  x_N ] = [ x4, x5, x6, x1, x2, x3 ]


Entry tableau
[[ 1.  0.  0.  0.  1. -2.  1.  0.]
 [ 0.  1.  0.  0.  1.  1.  1. 12.]
 [ 0.  0.  1.  0.  2.  1. -1.  6.]
 [ 0.  0.  0.  1. -1.  3.  0.  9.]]

k = 1 -> column no. 1 of x_N part ( x1 )
y_k
[ 1.  2. -1.]

r = 2 -> column no. 2 of x_B part ( x5 )
x_Br
3.0

pivot = 2.0

Pivoted tableau
[[ 1.   0.  -0.5  0.   0.  -2.5  1.5 -3. ]
 [ 0.   1.  -0.5  0.   0.   0.5  1.5  9. ]
 [ 0.   0.   0.5  0.   1.   0.5 -0.5  3. ]
 [ 0.   0.   0.5  1.   0.   3.5 -0.5 12. ]]

 x1 enters and  x5 leaves the basis


Iteration  2 
[ x_B ,  x_N ] = [ x4, x1, x6, x5, x2, x3 ]


Entry tableau
[[ 1.   0.  -0.5  0.   0.  -2.5  1.5 -3. ]
 [ 0.   1.  -0.5  0.   0.   0.5  1.5  9. ]
 [ 0.   0.   0.5  0.   1.   0.5 -0.5  3. ]
 [ 0.   0.   0.5  1.   0.   3.5 -0.5 12. ]]

k = 3 -> column no. 3 of x_N part ( x3 )
y_k
[ 1.5 -0.5 -0.5]

r = 1 -> column no. 1 of x_B part ( x4 )
x_Br
6.0

pivot = 1.5

Pivoted tableau
[[  1.      -1.       0.       0.      